In [2]:
%pip install pandas numpy matplotlib seaborn scipy scikit-learn imbalanced-learn kagglehub

## Import Dependences

In [3]:
import pandas as pd
import numpy as np

## Set display unlimited number of lines

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Read all datasets and save then in dataframes with the same name in a dictionary:

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("solarmainframe/ids-intrusion-csv")

print("Path to dataset files:", path)

100%|██████████| 1.60G/1.60G [00:40<00:00, 42.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/solarmainframe/ids-intrusion-csv/versions/1


## To preprocess the 02-20-2018.csv dataset together with all the others we need more than 30 GB of RAM, so to run this notebook in Kaggle this dataset will not be loaded in the dictionary and some code cells bellow will also be commented

In [6]:
import os

# Define the path to the folder containing the CSV files
folder_path = path

# Get a list of all the CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

dfs = dict()

# Loop through each CSV file and read it into a dataframe
for file in csv_files:
    df_name = os.path.splitext(file)[0]  # Get the dataframe name from the file name
    file_path = os.path.join(folder_path, file)  # Get the full file path
    df = pd.read_csv(file_path, low_memory=False)  # Read the CSV file into a dataframe
    locals()[df_name] = df  # Assign the dataframe to a variable with the same name as the file (e.g. 'data1 = df1')
    dfs[df_name] = df  # Store the dataframe in the dictionary with the same name as the file (e.g. 'dfs['data1'] = df1')

# Pre-processing

## 1. Drop Flow ID, Src IP and Dst IP columns of the 02-20-2018 dataset for being strings:

In [7]:
for id, df in dfs.items():
    try:
        dfs[id] = df.drop(['Flow ID','Src IP','Dst IP'], axis=1)
    except Exception:
        pass

## 2. Check existence and drop -inf and +inf values

In [8]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    count = df.values.ravel().tolist().count(np.inf)
    count = np.sum(df.values == np.inf)
    count2 = np.sum(df.values == -np.inf)
    print(f"Inf:'{count}'\n -Inf:'{count2}'\n")
    # replace +ve and -ve infinity with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop all NaN values
    df.dropna(inplace=True)

Dataframe: '02-16-2018'

Inf:'0'
 -Inf:'0'

Dataframe: '02-14-2018'

Inf:'5371'
 -Inf:'0'

Dataframe: '02-23-2018'

Inf:'7662'
 -Inf:'0'

Dataframe: '03-01-2018'

Inf:'0'
 -Inf:'0'

Dataframe: '02-28-2018'

Inf:'0'
 -Inf:'0'

Dataframe: '02-15-2018'

Inf:'11133'
 -Inf:'0'

Dataframe: '02-21-2018'

Inf:'0'
 -Inf:'0'

Dataframe: '02-20-2018'

Inf:'82139'
 -Inf:'0'

Dataframe: '03-02-2018'

Inf:'5542'
 -Inf:'0'

Dataframe: '02-22-2018'

Inf:'7651'
 -Inf:'0'



In [10]:
unique_attack = set()

for file_name, df in dfs.items():
    attack_kinds = df.Label.unique()
    unique_attack.update(attack_kinds)
    if 'Label' in attack_kinds:
        print("Attacks at", file_name, 'contain Label value(s)')
    print(f"Dataframe ['{file_name}'] has", len(attack_kinds), "attack kinds:", attack_kinds)

print(f"\n {len(unique_attack)} Unique attack kinds in CIC-IDS 2018:", unique_attack)

Attacks at 02-16-2018 contain Label value(s)
Dataframe ['02-16-2018'] has 4 attack kinds: ['Benign' 'DoS attacks-SlowHTTPTest' 'DoS attacks-Hulk' 'Label']
Dataframe ['02-14-2018'] has 3 attack kinds: ['Benign' 'FTP-BruteForce' 'SSH-Bruteforce']
Dataframe ['02-23-2018'] has 4 attack kinds: ['Benign' 'Brute Force -Web' 'Brute Force -XSS' 'SQL Injection']
Attacks at 03-01-2018 contain Label value(s)
Dataframe ['03-01-2018'] has 3 attack kinds: ['Benign' 'Label' 'Infilteration']
Attacks at 02-28-2018 contain Label value(s)
Dataframe ['02-28-2018'] has 3 attack kinds: ['Benign' 'Label' 'Infilteration']
Dataframe ['02-15-2018'] has 3 attack kinds: ['Benign' 'DoS attacks-GoldenEye' 'DoS attacks-Slowloris']
Dataframe ['02-21-2018'] has 3 attack kinds: ['Benign' 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC']
Dataframe ['02-20-2018'] has 2 attack kinds: ['Benign' 'DDoS attacks-LOIC-HTTP']
Dataframe ['03-02-2018'] has 2 attack kinds: ['Benign' 'Bot']
Dataframe ['02-22-2018'] has 4 attack kinds: ['Ben

Check columns of datasets

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    display(df.head())
    display(df.info())

Dataframe: '02-28-2018'



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,28/02/2018 08:22:13,94658,6,7,708,3718,387,0,...,20,0,0,0,0,0,0,0,0,Benign
1,443,6,28/02/2018 08:22:13,206,2,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Benign
2,445,6,28/02/2018 08:22:15,165505,3,1,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Benign
3,443,6,28/02/2018 08:22:16,102429,6,7,708,3718,387,0,...,20,0,0,0,0,0,0,0,0,Benign
4,443,6,28/02/2018 08:22:16,167,2,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Benign


<class 'pandas.core.frame.DataFrame'>
Index: 609063 entries, 0 to 613103
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Dst Port           609063 non-null  object
 1   Protocol           609063 non-null  object
 2   Timestamp          609063 non-null  object
 3   Flow Duration      609063 non-null  object
 4   Tot Fwd Pkts       609063 non-null  object
 5   Tot Bwd Pkts       609063 non-null  object
 6   TotLen Fwd Pkts    609063 non-null  object
 7   TotLen Bwd Pkts    609063 non-null  object
 8   Fwd Pkt Len Max    609063 non-null  object
 9   Fwd Pkt Len Min    609063 non-null  object
 10  Fwd Pkt Len Mean   609063 non-null  object
 11  Fwd Pkt Len Std    609063 non-null  object
 12  Bwd Pkt Len Max    609063 non-null  object
 13  Bwd Pkt Len Min    609063 non-null  object
 14  Bwd Pkt Len Mean   609063 non-null  object
 15  Bwd Pkt Len Std    609063 non-null  object
 16  Flow Byts/s        609063

None

Dataframe: '03-01-2018'



...

None

Dataframe: '02-23-2018'



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,6,23/02/2018 08:18:29,1532698,11,11,1179,1969,648,0,...,32,0.0,0.0,0,0,0.0,0.000000e+00,0,0,Benign
1,500,17,23/02/2018 08:17:45,117573855,3,0,1500,0,500,500,...,8,0.0,0.0,0,0,58786927.5,2.375324e+07,75583006,41990849,Benign
2,500,17,23/02/2018 08:17:45,117573848,3,0,1500,0,500,500,...,8,0.0,0.0,0,0,58786924.0,2.375325e+07,75583007,41990841,Benign
3,22,6,23/02/2018 08:19:55,1745392,11,11,1179,1969,648,0,...,32,0.0,0.0,0,0,0.0,0.000000e+00,0,0,Benign
4,500,17,23/02/2018 08:18:17,89483474,6,0,3000,0,500,500,...,8,4000364.0,0.0,4000364,4000364,21370777.5,1.528092e+07,41989576,7200485,Benign


<class 'pandas.core.frame.DataFrame'>
Index: 1042867 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1042867 non-null  int64  
 1   Protocol           1042867 non-null  int64  
 2   Timestamp          1042867 non-null  object 
 3   Flow Duration      1042867 non-null  int64  
 4   Tot Fwd Pkts       1042867 non-null  int64  
 5   Tot Bwd Pkts       1042867 non-null  int64  
 6   TotLen Fwd Pkts    1042867 non-null  int64  
 7   TotLen Bwd Pkts    1042867 non-null  int64  
 8   Fwd Pkt Len Max    1042867 non-null  int64  
 9   Fwd Pkt Len Min    1042867 non-null  int64  
 10  Fwd Pkt Len Mean   1042867 non-null  float64
 11  Fwd Pkt Len Std    1042867 non-null  float64
 12  Bwd Pkt Len Max    1042867 non-null  int64  
 13  Bwd Pkt Len Min    1042867 non-null  int64  
 14  Bwd Pkt Len Mean   1042867 non-null  float64
 15  Bwd Pkt Len Std    1042867 non-null  

None

Iterate trought all dictionary keys (in this case each key is a dataframe) and make a value count to see wich attacks there are in each day

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    count = df['Label'].value_counts()  # Perform value count on the 'A' column
    print(f"Value counts for dataframe '{key}':\n{count}\n")

Value counts for dataframe '02-28-2018':
Label
Benign           540568
Infilteration     68462
Label                33
Name: count, dtype: int64

Value counts for dataframe '03-01-2018':
Label
Benign           236632
Infilteration     92634
Label                25
Name: count, dtype: int64

Value counts for dataframe '02-16-2018':
Label
DoS attacks-Hulk            461912
Benign                      446772
DoS attacks-SlowHTTPTest    139890
Label                            1
Name: count, dtype: int64

Value counts for dataframe '02-15-2018':
Label
Benign                   988050
DoS attacks-GoldenEye     41508
DoS attacks-Slowloris     10990
Name: count, dtype: int64

Value counts for dataframe '02-21-2018':
Label
DDOS attack-HOIC        686012
Benign                  360833
DDOS attack-LOIC-UDP      1730
Name: count, dtype: int64

Value counts for dataframe '03-02-2018':
Label
Benign    758334
Bot       286191
Name: count, dtype: int64

Value counts for dataframe '02-22-2018':
Label
Be

## 3. Convert Timestamp values to pandas date and time datetime64 format

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

## 4. Convert datetime64 format to epoch format (epoch is Unix epoch time format, wich represents the number of seconds elapsed since January 1, 1970, at 00:00:00 UTC)

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    df['Timestamp'] = (df['Timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

## 5. Change all misclassified datatypes to float and substitute errors by NaN

In [ ]:
#Iterate throught each dataframe in the dictionary
for key in dfs.keys():
    df = dfs[key]
    for col in df.columns:
        #Check if the datatype of the column is object
        if df[col].dtype == 'object' and col != 'Label':
            # Change all values to numeric, and to NaN if it is a strig
            df[col] = pd.to_numeric(df[col], errors='coerce')



Count how many Na's there are in the dataframes:

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    count_NA = df.isna().sum()
    print(count_NA)

Dataframe: '02-28-2018'

Dst Port         33
Protocol         33
Timestamp        33
Flow Duration    33
Tot Fwd Pkts     33
                 ..
Idle Mean        33
Idle Std         33
Idle Max         33
Idle Min         33
Label             0
Length: 80, dtype: int64
Dataframe: '03-01-2018'

Dst Port         25
Protocol         25
Timestamp        25
Flow Duration    25
Tot Fwd Pkts     25
                 ..
Idle Mean        25
Idle Std         25
Idle Max         25
Idle Min         25
Label             0
Length: 80, dtype: int64
Dataframe: '02-16-2018'

Dst Port         1
Protocol         1
Timestamp        1
Flow Duration    1
Tot Fwd Pkts     1
                ..
Idle Mean        1
Idle Std         1
Idle Max         1
Idle Min         1
Label            0
Length: 80, dtype: int64
Dataframe: '02-15-2018'

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle 

Drop Na values

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    df.dropna(inplace=True)

## 6. Outliers Filtering

The objective of this filtering is removing extremely high values from the dataset that may have come from data noise

In [ ]:
import numpy as np
from scipy import stats

for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key


    # Define a function to filter outliers using Z-score
    def filter_outliers_zscore(data, threshold):
        z_scores = np.abs(stats.zscore(data))
        outlier_mask = (z_scores > threshold).any(axis=1)
        return data[~outlier_mask]

    # Define a threshold value
    threshold = 7

    # Loop through the columns of the dataframe and filter outliers in each column
    filtered_cols = []

    for col in df.columns:
        if col != 'Label':
            filtered_col = filter_outliers_zscore(df[[col]], threshold)
            filtered_cols.append(filtered_col)

    # Combine the filtered columns back into a dataframe
    df_filtered = pd.concat(filtered_cols, axis=1)
    display(df.head())
    display(df_filtered.head())


Script for calculating z_score percentage equivalent:

In [ ]:
from scipy.stats import norm
z_score= 7
percentage = (1- norm.cdf(z_score))*100

print("{:.10f}".format(percentage))

0.0000000001


Comparison between original and removed dataframes:

In [ ]:

from scipy import stats

# # Set the random seed for reproducibility
np.random.seed(134)

for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key

    # Define a function to filter outliers using Z-score
    def filter_outliers_zscore(data, threshold):
        z_scores = np.abs(stats.zscore(data))
        outlier_mask = (z_scores > threshold).any(axis=1)
        return data[~outlier_mask], data[outlier_mask]

    # Define a threshold value
    threshold = 7

    # The filtering was removing all DDoS-LOIC-UDP, so we will not execute it on them

    if key == '02-21-2018':
        df_temp = df[df['Label'] == 'DDOS attack-LOIC-UDP']
        df = df[df['Label'] != 'DDOS attack-LOIC-UDP']


    # Loop through the columns of the dataframe and filter outliers in each column
    filtered_cols = []
    removed_outliers = []
    for col in df.columns:
        if col != 'Label':
            filtered_col, outliers = filter_outliers_zscore(df[[col]], threshold)

            filtered_cols.append(filtered_col)
            removed_outliers.append(outliers)

    # Combine the filtered columns back into a dataframe
    df_filtered = pd.concat(filtered_cols, axis=1)

    # Combine the removed outliers back into a dataframe
    df_outliers = pd.concat(removed_outliers, axis=1)






    # Dataframe filtering comparison

    print(f'\nDataframe: {key} Shape: {df.shape}')

    # Print the number of outliers removed for each column

    print('Outlier removal summary:')
    n_outliers = df_outliers.shape[0]
    print(f'{n_outliers} outliers rows to be removed')

    # Print the original dataframe and the filtered dataframe side by side
    print('\nOriginal dataframe:')
    display(df.head())

    # Assign filtered dataframe columns to original one
    columns = [col for col in df.columns if col != 'Label']
    df.loc[:,columns] = df_filtered.loc[:,columns]


    # Recombine rows from 'DDOS attack-LOIC-UDP'
    if key == '02-21-2018':
        df = pd.concat([df,df_temp])


    print('\nFiltered dataframe:')
    display(df.head())

    # Print the removed outliers dataframe
    print('\nRemoved outliers:')
    display(df_outliers.head())

    # Count how many rows of each attack were removed from original dataframe
    values_orig = df.loc[df.index.isin(df_outliers.index), 'Label']
    print(f'\n{values_orig.value_counts()}')


/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1158: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - mean
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1158: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - mean



Dataframe: 02-28-2018 Shape: (609030, 80)
Outlier removal summary:
31452 outliers rows to be removed

Original dataframe:


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443.0,6.0,1.519806e+09,94658.0,6.0,7.0,708.0,3718.0,387.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,443.0,6.0,1.519806e+09,206.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,445.0,6.0,1.519806e+09,165505.0,3.0,1.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443.0,6.0,1.519806e+09,102429.0,6.0,7.0,708.0,3718.0,387.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443.0,6.0,1.519806e+09,167.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign



Filtered dataframe:


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443.0,6.0,1.519806e+09,94658.0,6.0,7.0,708.0,3718.0,387.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,443.0,6.0,1.519806e+09,206.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,445.0,6.0,1.519806e+09,165505.0,3.0,1.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443.0,6.0,1.519806e+09,102429.0,6.0,7.0,708.0,3718.0,387.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443.0,6.0,1.519806e+09,167.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


...

/tmp/ipython-input-269583654.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[11.  3.  3. ...  1. 11.  6.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:,columns] = df_filtered.loc[:,columns]
/tmp/ipython-input-269583654.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[11.  0.  0. ...  1. 12.  5.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:,columns] = df_filtered.loc[:,columns]
/tmp/ipython-input-269583654.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1179. 1500. 1500. ...   41. 1321.  455.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:,columns] = df_filtered.loc[:,columns]
/tmp/ipython-input-269583

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,6,1519373909,1532698,11.0,11.0,1179.0,1969.0,648.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Benign
1,500,17,1519373865,117573855,3.0,0.0,1500.0,0.0,500.0,NaN,...,8,0.0,0.0,0.0,0.0,58786927.5,NaN,75583006,41990849,Benign
2,500,17,1519373865,117573848,3.0,0.0,1500.0,0.0,500.0,NaN,...,8,0.0,0.0,0.0,0.0,58786924.0,NaN,75583007,41990841,Benign
3,22,6,1519373995,1745392,11.0,11.0,1179.0,1969.0,648.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Benign
4,500,17,1519373897,89483474,6.0,0.0,3000.0,0.0,500.0,NaN,...,8,4000364.0,0.0,4000364.0,4000364.0,21370777.5,NaN,41989576,7200485,Benign



Removed outliers:


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
5390,NaN,NaN,NaN,NaN,927.0,NaN,29453.0,NaN,NaN,NaN,...,200.0,NaN,11370730.0,NaN,11370730.0,11370730.0,NaN,NaN,NaN,NaN
5408,NaN,NaN,NaN,NaN,2627.0,5283.0,NaN,7604995.0,NaN,NaN,...,NaN,NaN,NaN,3.867811e+06,12271295.0,NaN,NaN,NaN,NaN,NaN
10930,NaN,NaN,NaN,NaN,2075.0,4009.0,NaN,5742019.0,NaN,NaN,...,115.0,NaN,10826702.0,NaN,10826702.0,10826702.0,NaN,NaN,NaN,NaN
19395,NaN,NaN,NaN,NaN,1294.0,22364.0,NaN,32591560.0,NaN,NaN,...,NaN,NaN,23958133.0,NaN,23958133.0,23958133.0,NaN,NaN,NaN,NaN
29302,NaN,NaN,NaN,NaN,2860.0,38417.0,NaN,56069495.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Label
Benign              35531
Brute Force -Web      190
Brute Force -XSS       77
SQL Injection           5
Name: count, dtype: int64


Count how many Na there are in the dataframes:

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    count_NA = df.isna().sum()
    print(count_NA)

Dataframe: '02-28-2018'

Dst Port            0
Protocol            0
Timestamp           0
Flow Duration       0
Tot Fwd Pkts       57
                 ... 
Idle Mean         142
Idle Std         3601
Idle Max          126
Idle Min          153
Label               0
Length: 80, dtype: int64
Dataframe: '03-01-2018'

Dst Port            0
Protocol            0
Timestamp           0
Flow Duration       0
Tot Fwd Pkts       76
                 ... 
Idle Mean           9
Idle Std         2361
Idle Max            0
Idle Min           14
Label               0
Length: 80, dtype: int64
Dataframe: '02-16-2018'

Dst Port            0
Protocol          177
Timestamp           0
Flow Duration    8800
Tot Fwd Pkts       36
                 ... 
Idle Mean        8795
Idle Std           19
Idle Max         8797
Idle Min         8795
Label               0
Length: 80, dtype: int64
Dataframe: '02-15-2018'

Dst Port            0
Protocol            0
Timestamp           0
Flow Duration       0
Tot Fwd Pkt

Drop Na values

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    df.dropna(inplace=True)

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(df.shape)

(577578, 80)
(310616, 80)
(1036539, 80)
(992169, 80)
(1048575, 80)
(1009488, 80)
(1011778, 80)
(7565845, 81)
(1011097, 80)
(1007064, 80)


## 7. Normalization

In [ ]:
for key in dfs.keys():
       df = dfs[key]  # Get the dataframe corresponding to the key
       print(f"Dataframe: '{key}'\n")
       display(df.describe())

Dataframe: '02-28-2018'



/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,577578.000000,577578.000000,5.775780e+05,5.775780e+05,577578.000000,577578.000000,577578.000000,577578.000000,577578.000000,577578.000000,...,577578.000000,577578.000000,5.775780e+05,5.775780e+05,5.775780e+05,5.775780e+05,5.775780e+05,5.775780e+05,5.775780e+05,5.775780e+05
mean,7170.355609,10.037747,1.519802e+09,1.074126e+07,4.461875,4.122764,334.883650,1437.932316,167.851861,15.214754,...,1.891744,15.994937,1.049528e+05,3.688948e+04,1.636528e+05,8.331021e+04,3.317246e+06,9.647041e+04,3.395919e+06,3.209352e+06
std,17006.779367,5.361862,1.420607e+04,3.062293e+07,8.361530,9.538951,861.060033,8144.815179,275.228847,20.553933,...,4.662804,6.672846,6.777868e+05,2.967516e+05,9.268822e+05,6.290109e+05,1.118967e+07,7.643169e+05,1.134299e+07,1.107022e+07
min,0.000000,0.000000,1.519780e+09,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,53.000000,6.000000,1.519787e+09,2.740000e+02,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,8.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,443.000000,6.000000,1.519808e+09,2.054000e+03,2.000000,1.000000,42.000000,86.000000,40.000000,0.000000,...,0.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2047.000000,17.000000,1.519815e+09,1.386482e+06,6.000000,5.000000,373.000000,355.000000,193.000000,36.000000,...,3.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,65533.000000,17.000000,1.519823e+09,1.200000e+08,1110.000000,450.000000,57019.000000,616364.000000,1968.000000,190.000000,...,1109.000000,44.000000,2.350000e+07,1.454452e+07,3.263753e+07,2.100000e+07,8.460000e+07,1.442817e+07,8.460000e+07,8.460000e+07


Dataframe: '03-01-2018'



/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


...

,Src Port,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,...,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06,7.565845e+06
mean,4.668125e+04,6.417406e+03,9.539141e+00,1.519111e+09,1.204838e+07,4.720830e+00,4.023582e+00,3.397468e+02,1.197943e+03,1.737333e+02,...,2.056324e+00,1.588454e+01,7.219001e+04,3.059514e+04,1.197300e+05,5.424224e+04,4.159707e+06,6.785571e+04,4.214756e+06,4.074360e+06
std,1.997864e+04,1.621929e+04,5.279353e+00,1.406687e+04,3.082939e+07,1.258477e+01,8.147632e+00,8.255895e+02,6.046546e+03,2.599331e+02,...,1.064437e+01,6.178594e+00,5.644627e+05,2.574867e+05,7.702044e+05,5.197388e+05,1.306867e+07,6.339755e+05,1.316521e+07,1.299133e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,1.519088e+09,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.995900e+04,5.300000e+01,6.000000e+00,1.519097e+09,5.110000e+02,1.000000e+00,1.000000e+00,2.000000e+01,0.000000e+00,2.000000e+01,...,0.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.270000e+04,8.000000e+01,6.000000e+00,1.519117e+09,5.176100e+04,2.000000e+00,1.000000e+00,4.400000e+01,1.140000e+02,4.200000e+01,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.815000e+04,3.389000e+03,1.700000e+01,1.519124e+09,2.830910e+06,7.000000e+00,6.000000e+00,4.360000e+02,9.640000e+02,1.940000e+02,...,3.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,6.553500e+04,1.700000e+01,1.519132e+09,1.200000e+08,8.164000e+03,9.970000e+02,2.612480e+05,1.201779e+06,1.968000e+03,...,8.163000e+03,4.800000e+01,2.281125e+07,1.401050e+07,3.056474e+07,1.928250e+07,1.082611e+08,1.286740e+07,1.082611e+08,1.082611e+08


Dataframe: '02-14-2018'



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,...,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06,1.011097e+06
mean,4.617869e+03,8.169613e+00,1.518595e+09,6.366756e+06,5.005059e+00,4.447752e+00,3.792241e+02,8.347496e+02,1.692143e+02,8.315534e+00,...,2.648352e+00,2.342916e+01,1.470823e+04,7.105414e+03,3.260628e+04,1.184884e+04,1.143227e+06,1.719559e+04,1.153016e+06,1.111306e+06
std,1.407674e+04,4.505407e+00,1.443707e+04,2.224621e+07,6.989601e+00,7.286993e+00,7.507051e+02,3.461690e+03,2.685747e+02,1.700098e+01,...,5.011792e+00,1.121421e+01,1.553650e+05,6.870820e+04,2.602018e+05,1.506151e+05,6.867112e+06,3.440432e+05,6.894340e+06,6.829427e+06
min,0.000000e+00,0.000000e+00,1.518570e+09,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.200000e+01,6.000000e+00,1.518578e+09,7.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.300000e+01,6.000000e+00,1.518602e+09,9.510000e+02,2.000000e+00,1.000000e+00,3.500000e+01,5.200000e+01,3.300000e+01,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,6.000000e+00,1.518607e+09,3.876120e+05,6.000000e+00,5.000000e+00,3.640000e+02,5.540000e+02,1.610000e+02,0.000000e+00,...,3.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553300e+04,1.700000e+01,1.518613e+09,1.200000e+08,2.650000e+02,4.600000e+02,2.769000e+04,5.432450e+05,1.460000e+03,1.370000e+02,...,4.000000e+01,4.800000e+01,4.067542e+06,1.554120e+06,5.243341e+06,3.959858e+06,1.199907e+08,7.021683e+07,1.199907e+08,1.199907e+08


Dataframe: '02-23-2018'



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,...,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06,1.007064e+06
mean,8.846537e+03,9.062825e+00,1.519371e+09,1.768982e+07,5.108689e+00,4.291095e+00,3.604430e+02,1.812014e+03,1.825204e+02,1.176497e+01,...,2.104868e+00,1.632117e+01,2.922885e+04,2.202510e+04,6.934826e+04,1.767343e+04,9.371842e+06,3.161354e+04,9.392247e+06,9.316843e+06
std,1.793467e+04,5.082191e+00,1.387377e+04,3.530063e+07,8.634089e+00,1.356504e+01,6.813042e+02,1.651594e+04,2.747218e+02,1.958987e+01,...,3.420717e+00,5.869909e+00,2.227867e+05,1.546356e+05,4.292287e+05,1.995038e+05,2.497839e+07,2.794278e+05,2.498881e+07,2.497285e+07
min,0.000000e+00,0.000000e+00,1.519348e+09,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,6.000000e+00,1.519357e+09,5.870000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.430000e+02,6.000000e+00,1.519376e+09,9.504750e+04,2.000000e+00,1.000000e+00,4.500000e+01,1.020000e+02,4.100000e+01,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.389000e+03,1.700000e+01,1.519383e+09,4.892688e+06,7.000000e+00,6.000000e+00,4.550000e+02,5.820000e+02,1.940000e+02,3.200000e+01,...,4.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553400e+04,1.700000e+01,1.519391e+09,1.199999e+08,7.510000e+02,1.489000e+03,1.549500e+04,1.984308e+06,1.968000e+03,2.110000e+02,...,3.200000e+01,4.800000e+01,5.896809e+06,3.639945e+06,9.103289e+06,5.147631e+06,1.199916e+08,1.128598e+07,1.199916e+08,1.199916e+08


Filter  inf and NaN again because some procedure added them again to 2 datasets:

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    # replace +ve and -ve infinity with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop all NaN values
    df.dropna(inplace=True)

Dataframe: '02-28-2018'

Dataframe: '03-01-2018'

Dataframe: '02-16-2018'

Dataframe: '02-15-2018'

Dataframe: '02-21-2018'

Dataframe: '03-02-2018'

Dataframe: '02-22-2018'

Dataframe: '02-20-2018'

Dataframe: '02-14-2018'

Dataframe: '02-23-2018'



In [ ]:
from sklearn.preprocessing import MinMaxScaler
for key in dfs.keys():
       df = dfs[key]  # Get the dataframe corresponding to the key
       columns = [col for col in df.columns if col != 'Label']
       min_max_scaler = MinMaxScaler().fit(df[columns])
       print(f"Dataframe: {key}\n")
       df[columns] = min_max_scaler.transform(df[columns])
       display(df.head())

Dataframe: 02-28-2018



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.00676,0.352941,0.614204,0.000789,0.004509,0.015556,0.012417,0.006032,0.196646,0.0,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,0.00676,0.352941,0.614204,0.000002,0.000902,0.000000,0.000000,0.000000,0.000000,0.0,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,0.00679,0.352941,0.614250,0.001379,0.001803,0.002222,0.000000,0.000000,0.000000,0.0,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,0.00676,0.352941,0.614273,0.000854,0.004509,0.015556,0.012417,0.006032,0.196646,0.0,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,0.00676,0.352941,0.614273,0.000001,0.000902,0.000000,0.000000,0.000000,0.000000,0.0,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


Dataframe: 03-01-2018



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1,0.0,0.0,0.611287,0.508312,0.001931,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.722749,0.0,0.722749,0.722749,Benign
3,0.0,0.0,0.614111,0.508313,0.001931,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.722749,0.0,0.722749,0.722749,Benign
7,0.0,0.0,0.622630,0.508313,0.001931,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.722749,0.0,0.722749,0.722749,Benign
8,0.0,0.0,0.625454,0.508313,0.001931,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.722749,0.0,0.722749,0.722749,Benign
9,0.0,0.0,0.628278,0.508313,0.001931,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.722749,0.0,0.722749,0.722749,Benign


Dataframe: 02-16-2018



...

,Src Port,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.694255,0.000336,0.352941,0.630732,0.007406,0.001225,0.011033,0.004781,0.001638,0.373984,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,Benign
1,0.000000,0.000000,0.000000,0.629691,0.938690,0.000245,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.520039,5.495334e-07,0.520039,0.520039,Benign
2,0.000000,0.000000,0.000000,0.633603,0.938689,0.000245,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.520039,1.428787e-06,0.520039,0.520039,Benign
3,0.000000,0.000000,0.000000,0.637515,0.938689,0.000245,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.520039,4.396267e-07,0.520039,0.520039,Benign
4,0.000000,0.000000,0.000000,0.641427,0.938689,0.000245,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.520039,5.055707e-06,0.520039,0.520039,Benign


Dataframe: 02-14-2018



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000000,0.000000,0.626427,0.938681,0.007576,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.469377,0.000002,0.469378,0.469376,Benign
1,0.000000,0.000000,0.630339,0.938679,0.007576,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.469376,0.000002,0.469376,0.469375,Benign
2,0.000000,0.000000,0.634251,0.938655,0.007576,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.469364,0.000004,0.469366,0.469362,Benign
3,0.000336,0.352941,0.639205,0.053783,0.053030,0.021739,0.044745,0.004184,0.509589,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Benign
4,0.000336,0.352941,0.639436,0.073367,0.049242,0.023913,0.041278,0.004066,0.509589,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Benign


Dataframe: 02-23-2018



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000336,0.352941,0.609019,0.012772,0.013333,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign
3,0.000336,0.352941,0.611010,0.014545,0.013333,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign
6,0.000000,0.000000,0.608926,0.938680,0.002667,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.469372,0.00001,0.469373,0.469372,Benign
7,0.000336,0.352941,0.613000,0.013053,0.013333,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign
12,0.000336,0.352941,0.614991,0.013425,0.016000,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign


## 8. Feature Correlation Filtering

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key


    columns = [col for col in df.columns if col != 'Label']

    corr_matrix = df[columns].corr().abs()

    threshold = 0.99
    # Find features with high correlation
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print(f'\nDataset: {key}')
    # Print features to drop
    print(f"The following {len(to_drop)} features will be dropped due to high correlation: {to_drop}")

    df = df.drop(to_drop, axis = 1)



Dataset: 02-28-2018
The following 16 features will be dropped due to high correlation: ['Fwd IAT Tot', 'Bwd Header Len', 'Pkt Len Min', 'Pkt Len Max', 'SYN Flag Cnt', 'ECE Flag Cnt', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Idle Mean', 'Idle Max', 'Idle Min']

Dataset: 03-01-2018
The following 16 features will be dropped due to high correlation: ['Fwd IAT Tot', 'Fwd IAT Max', 'Fwd IAT Min', 'Pkt Len Min', 'SYN Flag Cnt', 'ECE Flag Cnt', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Idle Mean', 'Idle Max', 'Idle Min']

Dataset: 02-16-2018
The following 36 features will be dropped due to high correlation: ['Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Std', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Bwd IAT Std'

Drop Na values

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    # replace +ve and -ve infinity with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop all NaN values
    df.dropna(inplace=True)

Dataframe: '02-28-2018'

Dataframe: '03-01-2018'

Dataframe: '02-16-2018'

Dataframe: '02-15-2018'

Dataframe: '02-21-2018'

Dataframe: '03-02-2018'

Dataframe: '02-22-2018'

Dataframe: '02-20-2018'

Dataframe: '02-14-2018'

Dataframe: '02-23-2018'



## 9. Equalization

In [ ]:
df = dfs['02-14-2018']

df = df.sample(frac=1) #Randomize rows's sequence


#df2 = df[df["Label"] == "FTP-BruteForce"][:193360]
#df3 = df[df["Label"] == "SSH-Bruteforce"][:187589]
df2 = df[df["Label"] == "FTP-BruteForce"]
df3 = df[df["Label"] == "SSH-Bruteforce"]
df0 = df[df["Label"] == "Benign"][:df2.shape[0]]
df1 = df[df["Label"] == "Benign"][:df3.shape[0]]


df_equal_FTP_BruteForce = pd.concat([ df0,df2], axis =0)
df_equal_SSH_BruteForce = pd.concat([ df1,df3], axis =0)

In [ ]:
df = dfs['02-15-2018']

df = df.sample(frac=1) #Randomize rows's sequence


df2 = df[df["Label"] == "DoS attacks-GoldenEye"]
df3 = df[df["Label"] == "DoS attacks-Slowloris"]
df0 = df[df["Label"] == "Benign"][:df2.shape[0]]
df1 = df[df["Label"] == "Benign"][:df3.shape[0]*100]


df_equal_DoS_GoldenEye = pd.concat([ df0,df2], axis =0)
df_equal_DoS_Slowloris = pd.concat([ df1,df3], axis =0)

In [ ]:
df = dfs['02-16-2018']

df = df.sample(frac=1) #Randomize rows's sequence



df3 = df[df["Label"] == "DoS attacks-SlowHTTPTest"]
df0 = df[df["Label"] == "Benign"]
df2 = df[df["Label"] == "DoS attacks-Hulk"][:df0.shape[0]]
df1 = df[df["Label"] == "Benign"][:df3.shape[0]]


df_equal_DoS_Hulk = pd.concat([ df0,df2], axis =0)
df_equal_DoS_SlowHTTPTest = pd.concat([ df1,df3], axis =0)

In [ ]:
df = dfs['02-21-2018']

df = df.sample(frac=1) #Randomize rows's sequence



df3 = df[df["Label"] == "DDOS attack-LOIC-UDP"]
df0 = df[df["Label"] == "Benign"]
df2 = df[df["Label"] == "DDOS attack-HOIC"][:df0.shape[0]]
df1 = df[df["Label"] == "Benign"][:df3.shape[0]*100]


df_equal_DDoS_HOIC = pd.concat([ df0,df2], axis =0)
df_equal_DDoS_LOIC_UDP = pd.concat([ df1,df3], axis =0)

In [ ]:
df_02_28 = dfs['02-28-2018']
df_03_01 = dfs['03-01-2018']

df_inf = pd.concat([ df_02_28,df_03_01], axis =0)

df_inf = df_inf.sample(frac=1) #Randomize rows's sequence


df2 = df_inf[df_inf["Label"] == "Infilteration"]
df1 = df_inf[df_inf["Label"] == "Benign"][:df2.shape[0]]


df_equal_Infilteration = pd.concat([ df1,df2], axis =0)

In [ ]:
df = dfs['03-02-2018']

df = df.sample(frac=1) #Randomize rows's sequence


df2 = df[df["Label"] == "Bot"]
df1 = df[df["Label"] == "Benign"][:df2.shape[0]]


df_equal_Bot = pd.concat([ df1,df2], axis =0)


In [ ]:
df = dfs['02-20-2018']

df = df.sample(frac=1) #Randomize rows's sequence

df2 = df[df["Label"] == "DDoS attacks-LOIC-HTTP"]
df1 = df[df["Label"] == "Benign"][:df2.shape[0]]

df_equal_DDoS_LOIC_HTTP = pd.concat([ df1,df2], axis =0)


Concat other dataframes that wont be equalized for being to unbalanced

In [ ]:
df1 = dfs['02-23-2018']
df2 = dfs['02-22-2018']
df_BruteForce_Web_XSS = pd.concat([df1,df2], axis = 0)
df_BruteForce_Web_XSS["Label"] = df_BruteForce_Web_XSS.Label.map(lambda a:"Benign" if a == 'Benign' else "Attack")

Add all equalized dataframes in a dictionary

In [ ]:
dfs_final = {}

# for name, df in zip(['df_equal_Bot','df_equal_DDoS_HOIC','df_equal_DDoS_LOIC_HTTP',
#                          'df_equal_DDoS_LOIC_UDP','df_equal_DoS_GoldenEye','df_equal_DoS_Hulk',
#                          'df_equal_DoS_SlowHTTPTest','df_equal_DoS_Slowloris','df_equal_FTP_BruteForce',
#                          'df_equal_Infilteration','df_equal_SSH_BruteForce','df_BruteForce_Web_XSS'],
#                          [df_equal_Bot,df_equal_DDoS_HOIC,df_equal_DDoS_LOIC_HTTP,
#                          df_equal_DDoS_LOIC_UDP,df_equal_DoS_GoldenEye,df_equal_DoS_Hulk,
#                          df_equal_DoS_SlowHTTPTest,df_equal_DoS_Slowloris,df_equal_FTP_BruteForce,
#                          df_equal_Infilteration,df_equal_SSH_BruteForce,df_BruteForce_Web_XSS]):
for name, df in zip(['df_equal_Bot','df_equal_DDoS_HOIC',
                         'df_equal_DDoS_LOIC_UDP','df_equal_DoS_GoldenEye','df_equal_DoS_Hulk',
                         'df_equal_DoS_SlowHTTPTest','df_equal_DoS_Slowloris','df_equal_FTP_BruteForce',
                         'df_equal_Infilteration','df_equal_SSH_BruteForce','df_BruteForce_Web_XSS'],
                         [df_equal_Bot,df_equal_DDoS_HOIC,
                         df_equal_DDoS_LOIC_UDP,df_equal_DoS_GoldenEye,df_equal_DoS_Hulk,
                         df_equal_DoS_SlowHTTPTest,df_equal_DoS_Slowloris,df_equal_FTP_BruteForce,
                         df_equal_Infilteration,df_equal_SSH_BruteForce,df_BruteForce_Web_XSS]):
    dfs_final[name] = df

In [ ]:
for key in dfs_final.keys():
    df = dfs_final[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    print(f"{df['Label'].value_counts()}\n")

Dataframe: 'df_equal_Bot'

Label
Benign    285763
Bot       285763
Name: count, dtype: int64

Dataframe: 'df_equal_DDoS_HOIC'

Label
Benign              360833
DDOS attack-HOIC    360833
Name: count, dtype: int64

Dataframe: 'df_equal_DDoS_LOIC_UDP'

Label
Benign                  173000
DDOS attack-LOIC-UDP      1730
Name: count, dtype: int64

Dataframe: 'df_equal_DoS_GoldenEye'

Label
Benign                   39924
DoS attacks-GoldenEye    39924
Name: count, dtype: int64

Dataframe: 'df_equal_DoS_Hulk'

Label
Benign              439126
DoS attacks-Hulk    439126
Name: count, dtype: int64

Dataframe: 'df_equal_DoS_SlowHTTPTest'

Label
Benign                      139890
DoS attacks-SlowHTTPTest    139890
Name: count, dtype: int64

Dataframe: 'df_equal_DoS_Slowloris'

Label
Benign                   272400
DoS attacks-Slowloris      2724
Name: count, dtype: int64

Dataframe: 'df_equal_FTP_BruteForce'

Label
Benign            193354
FTP-BruteForce    193354
Name: count, dtype: int64

Dataf

## 10. Save datasets

Final visualizations:

In [ ]:
for key in dfs_final.keys():
    df = dfs_final[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    display(df.describe())
    display(df.head())
    display(df.info())

Dataframe: 'df_equal_Bot'



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,...,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000,571526.000000
mean,0.126559,0.439652,0.516769,0.056957,0.005784,0.004048,0.019128,0.000758,0.133718,0.041016,...,0.014882,0.413794,0.001879,0.001283,0.002344,0.001412,0.023511,0.001698,0.023601,0.023266
std,0.191820,0.226567,0.348811,0.195393,0.011859,0.008927,0.032657,0.005670,0.164995,0.105003,...,0.027378,0.101823,0.024456,0.015144,0.022803,0.023309,0.115563,0.022242,0.115668,0.115423
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.006760,0.352941,0.181254,0.000004,0.001919,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.454545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.123295,0.352941,0.657816,0.000086,0.003839,0.001230,0.003868,0.000131,0.031507,0.000000,...,0.010204,0.454545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.123295,0.352941,0.857937,0.001766,0.003839,0.004920,0.021019,0.000163,0.223288,0.000000,...,0.010204,0.454545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,0.996161,0.771218,0.993681,0.781162,1.000000,1.000000,...,1.000000,0.909091,1.000000,0.995870,1.000000,1.000000,1.000000,0.990291,1.000000,1.000000


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
390810,0.423826,0.352941,0.248038,2.833334e-07,0.001919,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
847992,0.000809,1.000000,0.053219,2.209909e-03,0.001919,0.00246,0.003997,0.000279,0.021233,0.226277,...,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1038049,0.000809,1.000000,0.005208,1.527501e-05,0.000000,0.00123,0.004513,0.000201,0.047945,0.510949,...,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
370882,0.051714,0.352941,0.719415,2.052084e-02,0.013436,0.00861,0.073759,0.001609,0.463699,0.000000,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
858575,0.051714,0.352941,0.038635,3.081432e-02,0.013436,0.00861,0.073759,0.001609,0.463699,0.000000,...,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


<class 'pandas.core.frame.DataFrame'>
Index: 571526 entries, 390810 to 347458
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           571526 non-null  float64
 1   Protocol           571526 non-null  float64
 2   Timestamp          571526 non-null  float64
 3   Flow Duration      571526 non-null  float64
 4   Tot Fwd Pkts       571526 non-null  float64
 5   Tot Bwd Pkts       571526 non-null  float64
 6   TotLen Fwd Pkts    571526 non-null  float64
 7   TotLen Bwd Pkts    571526 non-null  float64
 8   Fwd Pkt Len Max    571526 non-null  float64
 9   Fwd Pkt Len Min    571526 non-null  float64
 10  Fwd Pkt Len Mean   571526 non-null  float64
 11  Fwd Pkt Len Std    571526 non-null  float64
 12  Bwd Pkt Len Max    571526 non-null  float64
 13  Bwd Pkt Len Min    571526 non-null  float64
 14  Bwd Pkt Len Mean   571526 non-null  float64
 15  Bwd Pkt Len Std    571526 non-null  float64
 16  Fl

None

...

Dataframe: 'df_BruteForce_Web_XSS'



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,...,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06,2.018842e+06
mean,1.316930e-01,5.372895e-01,5.484077e-01,1.477406e-01,7.344866e-03,3.922449e-03,1.777419e-02,1.160613e-03,9.123627e-02,5.708087e-02,...,6.045183e-02,3.533774e-01,5.652426e-03,8.268464e-03,9.102566e-03,3.779629e-03,7.714790e-02,1.990953e-03,7.749086e-02,7.651458e-02
std,2.706631e-01,3.010519e-01,3.241216e-01,2.950113e-01,1.476332e-02,1.133628e-02,3.661285e-02,9.761032e-03,1.373034e-01,9.371883e-02,...,9.898134e-02,1.290342e-01,4.107647e-02,5.554513e-02,5.382829e-02,3.986322e-02,2.064313e-01,2.176179e-02,2.068085e-01,2.062621e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.087528e-04,3.529412e-01,2.268571e-01,4.916667e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,1.818182e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.759850e-03,3.529412e-01,6.660802e-01,7.580046e-04,2.309469e-03,1.203369e-03,2.452404e-03,6.973217e-05,2.083333e-02,0.000000e+00,...,0.000000e+00,4.166667e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.171440e-02,1.000000e+00,8.375425e-01,4.143242e-02,9.333333e-03,4.813478e-03,2.330670e-02,4.419389e-04,9.603659e-02,1.516588e-01,...,1.052632e-01,4.545455e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000336,0.352941,0.609019,0.012772,0.013333,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign
3,0.000336,0.352941,0.611010,0.014545,0.013333,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign
6,0.000000,0.000000,0.608926,0.938680,0.002667,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.469372,0.00001,0.469373,0.469372,Benign
7,0.000336,0.352941,0.613000,0.013053,0.013333,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign
12,0.000336,0.352941,0.614991,0.013425,0.016000,0.007388,0.076089,0.000992,0.329268,0.0,...,0.666667,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,Benign


<class 'pandas.core.frame.DataFrame'>
Index: 2018842 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           float64
 1   Protocol           float64
 2   Timestamp          float64
 3   Flow Duration      float64
 4   Tot Fwd Pkts       float64
 5   Tot Bwd Pkts       float64
 6   TotLen Fwd Pkts    float64
 7   TotLen Bwd Pkts    float64
 8   Fwd Pkt Len Max    float64
 9   Fwd Pkt Len Min    float64
 10  Fwd Pkt Len Mean   float64
 11  Fwd Pkt Len Std    float64
 12  Bwd Pkt Len Max    float64
 13  Bwd Pkt Len Min    float64
 14  Bwd Pkt Len Mean   float64
 15  Bwd Pkt Len Std    float64
 16  Flow Byts/s        float64
 17  Flow Pkts/s        float64
 18  Flow IAT Mean      float64
 19  Flow IAT Std       float64
 20  Flow IAT Max       float64
 21  Flow IAT Min       float64
 22  Fwd IAT Tot        float64
 23  Fwd IAT Mean       float64
 24  Fwd IAT Std        float64
 25  Fwd IAT Max        floa

None

In [ ]:
combined = pd.concat(dfs_final.values())

Merge all attacks in one dataset for future algorithm analysis. There will be some null values because the datasets don't have the same number of columns. Those values will be replaced by NaN droped

In [ ]:
for column in combined.columns:
    if (combined[column] == 0).all():
        combined[column] = combined[column].replace(0,np.nan)

combined=combined.dropna(axis=1,how='all')

In [ ]:
combined.describe()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,...,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06,6.067376e+06
mean,2.236834e-01,3.914372e-01,3.956450e-01,1.047105e-01,6.395943e-02,4.106954e-02,6.450288e-02,2.140946e-02,2.005419e-01,3.674262e-02,...,5.786123e-02,4.514294e-01,3.182855e-03,4.058433e-03,4.847780e-03,2.301969e-03,4.321522e-02,1.504475e-03,4.348470e-02,4.280323e-02
std,3.510640e-01,3.061562e-01,3.596527e-01,2.314279e-01,1.464189e-01,1.013113e-01,1.644697e-01,5.524785e-02,3.036082e-01,8.691650e-02,...,1.044076e-01,1.806451e-01,3.137430e-02,3.873132e-02,3.919678e-02,3.046238e-02,1.661555e-01,2.130553e-02,1.665322e-01,1.659264e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.087528e-04,3.529412e-01,4.877811e-02,6.050001e-06,3.229682e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,4.166667e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.759850e-03,3.529412e-01,2.177365e-01,1.423436e-04,2.309469e-03,2.183406e-03,1.076763e-03,1.164134e-04,2.337398e-02,0.000000e+00,...,0.000000e+00,4.545455e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.576037e-01,3.529412e-01,7.947406e-01,3.252026e-02,2.078522e-02,1.612903e-02,2.101870e-02,3.467101e-03,3.683943e-01,0.000000e+00,...,5.102041e-02,6.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [ ]:
combined['Label'].value_counts()

,count
Label,
Benign,4263038
DoS attacks-Hulk,439126
DDOS attack-HOIC,360833
Bot,285763
FTP-BruteForce,193354
SSH-Bruteforce,187589
Infilteration,152861
DoS attacks-SlowHTTPTest,139890
DoS attacks-GoldenEye,39924


In [ ]:
df_std = combined.select_dtypes(include=np.number).std()
display(df_std.sort_values())

,0
FIN Flag Cnt,0.000406
Idle Std,0.021306
Bwd IAT Min,0.027300
Active Min,0.030462
Active Mean,0.031374
...,...
Init Bwd Win Byts,0.368085
ECE Flag Cnt,0.394477
RST Flag Cnt,0.394477
ACK Flag Cnt,0.491794


In [ ]:
combined.shape

(6067376, 70)

In [ ]:
output_dir = 'cicids2018_preprocessed.parquet'
combined.to_parquet(output_dir)

## The datasets output are preprocessed and ready to use in ML modeling